In [1]:
import cv2
import numpy as np

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [2]:
# Initialize Detector & load pre-trained weight
prototxt_path = "./caffe_dnn_module/deploy.prototxt.txt"
weights_path = "./caffe_dnn_module/res10_300x300_ssd_iter_140000.caffemodel"

detector = cv2.dnn.readNet(prototxt_path, weights_path)

# Load pre-trained CNN model
CNN = load_model("./checkpoint/Mobilenet_2")


In [33]:
test_img = cv2.imread("./data/testdata/1.jpg")


In [34]:
blob = cv2.dnn.blobFromImage(test_img, 1.0, (300, 300),
                             (104.0, 177.0, 123.0))
(h, w) = test_img.shape[:2]

detector.setInput(blob)
detections = detector.forward()
print(max(detections[0,0,:,2]))

0.23369686


In [35]:
threshold = 0.5
for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]

    if confidence > threshold:
        box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
        (startX, startY, endX, endY) = box.astype("int")
        print(confidence, startX, startY, endX, endY)

        text = "{:2f}%".format(confidence * 100)
        y = startY - 10 if startY - 10 > 10 else startY + 10
        cv2.rectangle(test_img, (startX, startY), (endX, endY), (0, 255, 0), 2)
        cv2.putText(test_img, text, (startX, y),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

cv2.imshow("result", test_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

